In [ ]:
import os
import numpy as np
import pandas as pd
import selfies as sf
import matplotlib.pyplot as plt
import seaborn as sns
from textdistance import levenshtein
from tqdm import tqdm

In [ ]:
output_dir = 'outputs_3_levenshtein_distance_analysis'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [ ]:
## baseline
filepath_base = os.path.join('baseline', 'GA+D.txt')
generated_base = pd.read_csv(filepath_base, header=None).iloc[:,0].values.tolist()
print(len(generated_base))

In [ ]:
## frag-level
filepath_frag = os.path.join('outputs_2_generate_molecules', 'zinc15+frag', 'smi_after.csv')
generated_frag = pd.read_csv(filepath_frag, header=None).iloc[:,0].values.tolist()
print(len(generated_frag))

In [ ]:
## char-level
filepath_char = os.path.join('outputs_2_generate_molecules', 'zinc15+char', 'smi_after.csv')
generated_char = pd.read_csv(filepath_char, header=None).iloc[:,0].values.tolist()
print(len(generated_char))

In [ ]:
reconstructed_base = [sf.encoder(sf.decoder(x)) for x in generated_base]
reconstructed_frag = [sf.encoder(sf.decoder(x)) for x in generated_frag]
reconstructed_char = [sf.encoder(sf.decoder(x)) for x in generated_char]

In [ ]:
base_gen = [list(sf.split_selfies(x)) for x in generated_base]
frag_gen = [list(sf.split_selfies(x)) for x in generated_frag]
char_gen = [list(sf.split_selfies(x)) for x in generated_char]

In [ ]:
base_rec = [list(sf.split_selfies(x)) for x in reconstructed_base]
frag_rec = [list(sf.split_selfies(x)) for x in reconstructed_frag]
char_rec = [list(sf.split_selfies(x)) for x in reconstructed_char]

In [ ]:
base_sim = [levenshtein.normalized_similarity(x, y) for x, y in zip(base_gen, base_rec)]
frag_sim = [levenshtein.normalized_similarity(x, y) for x, y in zip(frag_gen, frag_rec)]
char_sim = [levenshtein.normalized_similarity(x, y) for x, y in zip(char_gen, char_rec)]

In [ ]:
base_dist = [levenshtein.normalized_distance(x, y) for x, y in zip(base_gen, base_rec)]
frag_dist = [levenshtein.normalized_distance(x, y) for x, y in zip(frag_gen, frag_rec)]
char_dist = [levenshtein.normalized_distance(x, y) for x, y in zip(char_gen, char_rec)]

In [ ]:
data = pd.DataFrame({
    'Normalized Levenshtein Distance':frag_dist + base_dist,
    'Degree of Collapse':([f'ReBADD-SE (frag): {np.mean(frag_dist):.2f}'] * len(frag_dist))
                       + ([f'GA+D: {np.mean(base_dist):.2f}'] * len(base_dist))
})

In [ ]:
sns.set_theme(style='darkgrid')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6, 3))

_ = sns.histplot(data=data, x='Normalized Levenshtein Distance', binwidth=0.05, hue='Degree of Collapse',
                 kde=True, common_norm=False, ax=ax)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'selfies_collapse_analysis.png'), dpi=600)